In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="../.env")

model = ChatOpenAI(
    base_url=os.getenv("MODEL_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model=os.getenv("MODEL_NAME", "gpt-3.5-turbo"),
    temperature=0
)

In [8]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent

@dataclass
class Context:
    user_id: str

@tool
def fetch_user_email_preferences(tool_runtime: ToolRuntime) -> str:
    """Fetch the user's email preferences from the store."""
    user_id = tool_runtime.context.user_id

    preferences: str = "The user prefers you to write a brief and polite email."
    if tool_runtime.store:
        if memory := tool_runtime.store.get(("users",), user_id):
            preferences = memory.value["preferences"]

    return preferences

@tool
def set_user_email_preferences(tool_runtime: ToolRuntime, preferences: str) -> str:
    """Set the user's email preferences in the store."""
    user_id = tool_runtime.context.user_id

    if memory := tool_runtime.store.get(("users",), user_id):
        user_data = memory.value
        user_data["preferences"] = preferences
        tool_runtime.store.put(("users",), user_id, user_data)
    else:
        user_data = {"preferences": preferences}
        tool_runtime.store.put(("users",), user_id, user_data)

    return f"Successfully updated email preferences for user {user_id}: {preferences}"

agent = create_agent(
    model=model,
    tools=[fetch_user_email_preferences, set_user_email_preferences],
    context_schema=Context,
)

In [9]:
agent.invoke(
    {"messages": [{"role": "user", "content": "I prefer to write affectionate and medium-length emails."}]},
    context=Context(user_id="John Smith")
)

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Please write an email to LiLei far away to ask how she is doing recently."}]},
    context=Context(user_id="John Smith")
)
print(response)